In [3]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DDPG_single_lane import DDPGagent_single_lane
import traci
import traci.constants as tc
import math
import torch
import torch.autograd
import torch.optim as optim
import torch.nn as nn
from collections import deque
import random
from model_single_lane import *
from utils_single_lane import *
from memory import *

In [4]:
import os, sys
tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
sys.path.append(tools)

In [14]:
import pandas
import pickle
filename = 'actor2_230_single_lane_collide.sav'
actor1=pickle.load(open(filename, 'rb'))

In [20]:
#test the model-collide final seed-20000 random-false
r=36.61
rl1='autonomous_vehicle'
num_veh=21

for episode in range(1):

    traci.start(["sumo-gui", "-c", "../maps1/ring_example_20190424-2048001556153280.685167.sumo.cfg"])
    traci.simulationStep()
    vehID=traci.vehicle.getIDList()
    while len(vehID)!=num_veh:
        traci.close()
        traci.start(["sumo-gui", "-c", "../maps1/ring_example_20190424-2048001556153280.685167.sumo.cfg"])
        traci.simulationStep()
        vehID=traci.vehicle.getIDList()
        
        
    for j in range(len(vehID)):
        testVname=vehID[j]
            
        vehposition=traci.vehicle.getPosition(testVname)
        vehpos_1=vehposition[0]-r
        vehpos_2=vehposition[1]-r
        
        if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
    
        if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
        if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
        if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
        
        
# reset state
    for reset_state in range(1000):
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
            
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
        
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
    
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')

            traci.vehicle.subscribe(testVname,(tc.VAR_SPEED,tc.VAR_POSITION))

            
    vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(vehID[0])
    speed_rl1=vehicle_obs_rl1[tc.VAR_SPEED]
    headway_total=traci.vehicle.getLeader(rl1,200)
    veh_leader=headway_total[0]
    headway=headway_total[1]+2
    speed_leader=traci.vehicle.getSpeed(veh_leader)

    state=np.array([speed_rl1,speed_leader,headway])
                
    speed_total_e=speed_rl1
    for i in range(1,len(vehID)):
        vehicle_obs_e=traci.vehicle.getSubscriptionResults(vehID[i])
        veh_e=vehicle_obs_e[tc.VAR_SPEED]
        speed_total_e=np.hstack((speed_total_e,veh_e))
    speed_total_e_01=speed_total_e       
    
#start training
    traci.vehicle.setSpeedMode("autonomous_vehicle",0)
    for step in range(2071):
   
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action=actor1(state_01)
        action = action.detach().numpy()[0]        
 #      new_state, reward, done, _ = env.step(action) 
 # accelerating
        acc=action
        Speed_e=acc*0.1+state[0]
        Speed=max(Speed_e,0)
        traci.vehicle.slowDown(rl1,Speed,0.1)

# done
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
            
            traci.vehicle.subscribe(testVname,(tc.VAR_SPEED,tc.VAR_POSITION))

        vehID=traci.vehicle.getIDList()
        
        done = 0
# new_state and reward
        
        if done==0:
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(rl1)
            
            speed_rl1=vehicle_obs_rl1[tc.VAR_SPEED]
            headway_total=traci.vehicle.getLeader(rl1,200)
            veh_leader=headway_total[0]
            headway=headway_total[1]+2
            speed_leader=traci.vehicle.getSpeed(veh_leader)

            new_state=np.array([speed_rl1,speed_leader,headway])    
        
        state = new_state
            
    traci.vehicle.setSpeedMode("malicious_vehicle",0)
    for step in range(2071,2071+10):
   
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action=actor1(state_01)
        action = action.detach().numpy()[0]        

 # accelerating
        acc=action
        Speed_e=acc*0.1+state[0]
        Speed=max(Speed_e,0)
        traci.vehicle.slowDown(rl1,Speed,0.1)
        acc_v13=-4
        Speed_v13_e=acc_v13*0.1+state[1]
        Speed_v13=max(Speed_v13_e,2.2)
        traci.vehicle.slowDown("malicious_vehicle",Speed_v13,0.1)

# done
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
            
            traci.vehicle.subscribe(testVname,(tc.VAR_SPEED,tc.VAR_POSITION))

        vehID=traci.vehicle.getIDList()
        
        done = 0
# new_state and reward
        
        if done==0 or step==2999:
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(rl1)
            
            speed_rl1=vehicle_obs_rl1[tc.VAR_SPEED]
            headway_total=traci.vehicle.getLeader(rl1,200)
            veh_leader=headway_total[0]
            headway=headway_total[1]+2
            speed_leader=traci.vehicle.getSpeed(veh_leader)

            new_state=np.array([speed_rl1,speed_leader,headway])
       
        state = new_state
        episode_reward += reward
        
        
    traci.vehicle.setSpeedMode("malicious_vehicle",0)
    for step in range(2071+10,2071+20):
   
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action=actor1(state_01)
        action = action.detach().numpy()[0]         
 # accelerating
        acc=action
        Speed_e=acc*0.1+state[0]
        Speed=max(Speed_e,0)
        traci.vehicle.slowDown(rl1,Speed,0.1)
        traci.vehicle.slowDown("malicious_vehicle",2.2,0.5)

# done
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
            
            traci.vehicle.subscribe(testVname,(tc.VAR_SPEED,tc.VAR_POSITION))

        vehID=traci.vehicle.getIDList()

# new_state and reward
        done=0       
        if done==0 or step==2999:
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(rl1)
            
            speed_rl1=vehicle_obs_rl1[tc.VAR_SPEED]
            headway_total=traci.vehicle.getLeader(rl1,200)
            veh_leader=headway_total[0]
            headway=headway_total[1]+2
            speed_leader=traci.vehicle.getSpeed(veh_leader)

            new_state=np.array([speed_rl1,speed_leader,headway])    
        
        state = new_state
        
    traci.vehicle.setSpeedMode("malicious_vehicle",31)
    for step in range(2071+20,3000):
   
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action=actor1(state_01)
        action = action.detach().numpy()[0]        
 #      new_state, reward, done, _ = env.step(action) 
 # accelerating
        acc=action
        Speed_e=acc*0.1+state[0]
        Speed=max(Speed_e,0)
        traci.vehicle.slowDown(rl1,Speed,0.1)

# done
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
            
            traci.vehicle.subscribe(testVname,(tc.VAR_SPEED,tc.VAR_POSITION))

        vehID=traci.vehicle.getIDList()

# new_state and reward
        done=0
        if done==0 or step==2999:
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(rl1)
            
            speed_rl1=vehicle_obs_rl1[tc.VAR_SPEED]
            headway_total=traci.vehicle.getLeader(rl1,200)
            veh_leader=headway_total[0]
            headway=headway_total[1]+2
            speed_leader=traci.vehicle.getSpeed(veh_leader)

            new_state=np.array([speed_rl1,speed_leader,headway])
        
        state = new_state
        episode_reward += reward

    traci.close()


 Retrying in 1 seconds


FatalTraCIError: connection closed by SUMO